In [1]:
pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


2024-06-17 22:12:20.587535: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 22:12:20.587687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 22:12:20.790371: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def question_answer(question, reference):
    """
    using bert to answer some question
    """

    model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")
    tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

    tokens = ['[CLS]'] + tokenizer.tokenize(question) + ['[SEP]'] + tokenizer.tokenize(reference) + ['[SEP]']
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    mask = [1] * len(input_ids)
    # apply mask,0 for reference and 1 for questions
    type_ids = [0] * (1 + len(tokenizer.tokenize(question)) + 1) + [1] * (len(tokenizer.tokenize(reference)) + 1)
    input_ids, mask, type_ids = map(lambda f: tf.expand_dims(
        tf.convert_to_tensor(f, dtype=tf.int32), 0),
                                   (input_ids, mask, type_ids))

    outputs = model([input_ids, mask, type_ids])
    

    short_start = tf.argmax(outputs[0][0][1:]) + 1
    short_end = tf.argmax(outputs[1][0][1:]) + 1
    if short_start <= len(tokenizer.tokenize(question)) or short_start >= short_end:
        return None

    answer_tokens = tokens[short_start: short_end + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    return answer

In [4]:
def semantic_search(corpus_path, sentence):
    """Perform semantic search using SentenceTransformer and cosine similarity."""
    
    # Load SentenceTransformer model
    model = SentenceTransformer("all-MiniLM-L6-v2")
    
    corpus = []
    file_paths = []
    
    for filename in os.listdir(corpus_path):
        if filename.endswith(".md"):
            file_path = os.path.join(corpus_path, filename)
            with open(file_path, 'r') as md_file:
                corpus.append(md_file.read())
                file_paths.append(file_path)
    
    corpus_embeddings = model.encode(corpus)
    
    sentence_embedding = model.encode(sentence).reshape(1, -1)
    
    similarities = cosine_similarity(sentence_embedding, corpus_embeddings).flatten()
    
    most_similar_idx = np.argmax(similarities)
    
    most_similar_doc = corpus[most_similar_idx]
    
    return most_similar_doc

In [5]:
def answer_loop(coprus_path):
    """loop to answer questions from a reference text"""
    while True:
        try:
            question = input("Q: ")
            question = question.lower()
            if question in ['exit', 'quit', 'goodbye', 'bye']:
                print("A: Goodbye")
                sys.exit()
            else:
                reference = semantic_search(coprus_path, question)
                if question_answer(question, reference) == None or question_answer(question, reference) == "":
                    print("A: Sorry, I do not understand your question.")
                else:
                    print("A: {}".format(question_answer(question, reference)))
        except (KeyboardInterrupt, EOFError):
            print("A: Goodbye")
            sys.exit()

In [8]:
answer_loop('/kaggle/input/all-qa-data')

Q:  When are PLDs?


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

A: Sorry, I do not understand your question.


Q:  EXIT


A: Goodbye


NameError: name 'sys' is not defined